## Disponível online

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/carlos-adir/UnB-Courses/blob/dev/mntf/fem-nurbsnD.ipynb)

Esse python notebook está disponível online no GitHub através do link:

* [GitHub/carlos-adir/UnB-Courses/mntf/fem-nurbsnD](https://github.com/carlos-adir/UnB-Courses/blob/dev/mntf/fem-nurbsnD.ipynb)

In [ ]:
import os
os.system("pip install numpy")
# os.system("pip install pandas")
os.system("pip install mpmath")
os.system("pip install matplotlib")
os.system("pip install compmec-nurbs")
import numpy as np
import sympy as sp
import math
import mpmath
from typing import Callable, Iterable, Optional, Tuple
from matplotlib import pyplot as plt
from compmec import nurbs  
from helper import getH, getD, solve_system, plot_field

# Teoria

Até então fizemos o Método dos Elementos Finitos (FEM) usando BSplines para 1D. Ou seja, até então tratamos EDOs.

### Recapitulação sobre BSplines

Recapitulando, usamos funções de base BSplines com um vetor de nós $U$:

$$
U = \left[u_0, \ u_1, \ \cdots, \ u_p, \ u_{p+1}, \ \cdots, \ u_{n-1}, \ u_{n}, \ \cdots, \ u_{n+p}\right]
$$
$$
0 = u_0 = u_1 = \cdots = u_{p} < u_{p+1} \le \cdots \le u_{n-1} < u_{n} = \cdots = u_{n+p} = 1
$$

Em que $p$ é a ordem do polinômio. Desta forma, teremos $p+1$ vetores de $n$ funções:

$$
\left[N_{\text{all}}\right] = 
\begin{bmatrix}
N_{00}(u) & N_{10}(u) & \cdots & N_{n-1, 0}(u) \\ 
N_{01}(u) & N_{11}(u) & \cdots & N_{n-1, 1}(u) \\ 
\vdots & \vdots & \ddots & \vdots \\ 
N_{0p}(u) & N_{1p}(u) & \cdots & N_{n-1, p}(u) \\ 
\end{bmatrix}
$$

Uma notação que usamos é que

$$
\left[N_{j}\right] = \begin{bmatrix}N_{0j} & N_{1j} & \cdots & N_{n-2,j} & N_{n-1,j}\end{bmatrix} \ \ \ \ \ \ \ \forall \ 0 \le j \le p
$$

Como $N_{ij}(u) = 0 \ \forall i=\left\{0, \ \cdots, \ p-j\right\}$, então podemos substituir e escrever

$$
\left[N_{\text{all}}\right] = 
\begin{bmatrix}
0 & 0 & 0 & \cdots & 0 & 0 & N_{p,0} & \cdots & N_{n-1, 0} \\ 
0 & 0 & 0 & \cdots & 0 & N_{p-1, 1} & N_{p,1} & \cdots & N_{n-1, 1} \\ 
0 & 0 & 0 & \cdots & N_{p-2, 2} & N_{p-1, 1} & N_{p,1} & \cdots & N_{n-1, 1} \\ 
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \ddots & \vdots  \\ 
0 & 0 & N_{2,p-2} & \cdots & N_{p-2,p-2} & N_{p-1,p-2} & N_{p,p-2} & \cdots & N_{n-1, p} \\ 
0 & N_{1,p-1} & N_{2,p-1} & \cdots & N_{p-2,p-1} & N_{p-1,p-1} & N_{p,p-1} & \cdots & N_{n-1, p} \\ 
N_{0p} & N_{1p} & N_{2p} & \cdots & N_{p-2,p} & N_{p-1,p} & N_{p,p} & \cdots & N_{n-1, p} \\ 
\end{bmatrix}
$$

Assim, a solução que procuramos é descrita da forma

$$
w(u) = \sum_{i=0}^{n-1} w_{i} \cdot N_{ip}(u) = \left[N_p\right] \cdot \left[w\right]
$$

Como lidamos com derivadas, vimos também que a derivada se comporta da forma:

$$
\dfrac{dw}{du} = \left[\dfrac{dN_p}{du}\right] \cdot \left[w\right] = \left[D_p\right] \cdot \left[N_{p-1}\right] \cdot \left[w\right]
$$
$$
\dfrac{d^2w}{du^2} = \left[\dfrac{d^2N_p}{du^2}\right] \cdot \left[w\right] = \left[D_p\right] \cdot \left[D_{p-1}\right] \cdot \left[N_{p-2}\right] \cdot \left[w\right]
$$

Em que 

$$\alpha_{ij} =  \begin{cases} \dfrac{j}{u_{i+j}-u_{i}} \ \ \ \text{if} \ u_{i} \ne u_{i+j} \\
0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \text{else}\end{cases}$$
$$\left[D_{j}\right] = \begin{bmatrix}
\alpha_{0j} & -\alpha_{1j} & & & \\
& \alpha_{1j} & -\alpha_{2j} & & \\
& & \alpha_{2j} & -\alpha_{3j} & \\
& & & \ddots & \ddots & \\
& & & & \alpha_{n-1,j} & -\alpha_{n,j} \\
& & & & & \alpha_{nj}
\end{bmatrix}$$

Para elementos finitos é importante integrar, de forma que 

$$
\left[H_{yz}\right]_{n \times n} = \int_{0}^{1} \left[N_{y}\right] \otimes \left[N_{z}\right] \ du = \sum_{k=p}^{n-1} \left[H_{yz}^{(k)}\right]
$$
$$
\left[H_{yz}^{(k)}\right]_{(n \times n)} = \int_{u_k}^{u_{k+1}} \left[N_{y}\right] \otimes \left[N_{z}\right] \ du = \left(u_{k+1}-u_{k}\right)
\begin{bmatrix} \left[0\right] & \left[0\right] & \left[0\right] \\ \left[0\right] & \left[M_{yz}^{(k)}\right] & \left[0\right] \\ \left[0\right] & \left[0\right] & \left[0\right] \end{bmatrix}
$$
$$
\left[M_{yz}^{(k)}\right]_{(y+1)\times(z+1)} = \dfrac{1}{u_{k+1}-u_{k}} \int_{u_k}^{u_{k}}
\begin{bmatrix}
N_{k-y,y}N_{k-z,z} & N_{k-y,y}N_{k-z+1,z} & \cdots & N_{k-y,y}N_{k,z} \\ 
N_{k-y+1,y}N_{k-z,z} & N_{k-y+1,y}N_{k-z+1,z} & \cdots & N_{k-y+1,y}N_{k,z} \\ 
\vdots & \vdots & \cdots & \vdots \\ 
N_{k,y}N_{k-z,z} & N_{k,y}N_{k-z+1,z} & \cdots & N_{k,y}N_{k,z} \\ 
\end{bmatrix} \ du
$$

Para valores de $y$ e $z$ pequenos, teremos:

$$
\left[M_{00}^{(k)}\right] = \begin{bmatrix} 1 \end{bmatrix}
$$
$$
\left[M_{01}^{(k)}\right] = \dfrac{1}{2}\begin{bmatrix} 1 & 1 \end{bmatrix}
$$
$$
\left[M_{11}^{(k)}\right] = \dfrac{1}{6}\begin{bmatrix} 2 & 1 \\ 1 & 2 \end{bmatrix}
$$

Já para valores maiores, é necessário investigar, mas já existe uma função ```getMatrix(y, z, k, N)``` que já entrega o valor de $[M_{yz}]$ usando integrais numéricas.

# Aplicação FEM com BSplines na equação de Poisson

Agora, seja a equação de Poisson dada por

$$
\dfrac{\partial T}{\partial t} =  \mu \dfrac{\partial^2 T}{\partial x^2} + f(t, x) \ \ \ \ \ \ \ \ \ \text{on} \ \Omega
$$

Com $\mu \in \mathbb{R}$ e dominio retangular $$\Omega = \underbrace{\left[0, \ 1 \right]}_{t} \times \underbrace{\left[0, \ 1\right]}_{x}$$.

Queremos encontrar uma solução aproximada da forma

$$
T(t, x) = \sum_{m=0}^{n_t-1} \sum_{i=0}^{n_{x}-1} N_{m,p_t}(t) \cdot N_{i,p_{x}}(x) \cdot T_{m,i}
$$

Com $n_t \times n_x$ valores $T_{ij}$ desconhecidos.
Assim, aplicando na formulação de elementos finitos:

$$
\int_{\Omega} \left(\dfrac{\partial T}{\partial t} - \alpha \dfrac{\partial^2 T}{\partial x^2} - f\right) \cdot \varphi \ d\Omega  \ \ \ \ \ \ \ \ \forall \varphi
$$

Obtemos o sistema linear:

$$
\sum_{r=0}^{n_t-1} \sum_{s=0}^{n_x-1} A_{m,r,i,s} \cdot T_{r,s} = B_{m,i} \ \ \ \ \  \forall m,  \ i
$$

$$
A_{m, r, i, s} = \int_{\Omega} N_{m,p_t} N_{i,p_x} \left(\dfrac{dN_{r,p_t}}{dt}N_{s,p_x} - \mu N_{r,p_t} \dfrac{d^2 N_{s,p_x}}{dx^2}\right) \ d\Omega
$$
$$
B_{m, i} = \int_{\Omega} N_{m,p_t}N_{i,p_x} f \ d\Omega
$$

A inicio as contas podem parecer complicadas, mas basta um pouco de trabalho:

> Para a matriz $A$, vemos que as integrais são separáveis:
> 
> $$A_{m,r,i,s} = \left(\int_{0}^{1} N_{m,p_t} \frac{dN_{r,p_t}}{dt}\ dt\right) \left(\int_{0}^{1} N_{i,p_x} N_{s,p_x}\ dx\right) - \mu \left(\int_{0}^{1} N_{m,p_t} N_{r,p_t}\ dt\right)\left(\int_{0}^{1} N_{i,p_x} \frac{d^2N_{s,p_x}}{dx^2}\ dx\right) $$
> 
> Que já vimos anteriormente que essas integrais são já conhecidas do caso 1D:
> 
> $$\int_{0}^{1} N_{m,p_t}(t)N_{r,p_t}(t) \ dt = \left[H_{p_t,p_t}\right]_{m,r}$$
> $$\int_{0}^{1} N_{i,p_x}(x)N_{s,p_x}(x) \ dx = \left[H_{p_x,p_x}\right]_{i,s}$$
> $$\int_{0}^{1} N_{m,p_t}(t)\dfrac{dN_{r,p_t}}{dt}(t) \ dt = \left(\left[H_{p_t-1,p_t}\right]^{T}\left[D_{p_t}\right]^{T}\right)_{m,r}$$
> $$\int_{0}^{1} N_{i,p_x}(x)\dfrac{d^2N_{s,p_x}}{dx^2} \ dx = \left( \left[X_{bound}\right] \left[D_{p_x}\right]^{T} - \left[D_{p_x}\right] \left[H_{p_x-1,p_x-1}\right] \left[D_{p_x}\right]^{T}\right)_{i,s}$$
> $$X_{bound} = \left(\left[N_{p_{x}}\right] \otimes \left[N_{p_x-1}\right]\right)_{0}^{1}$$
> 
> Logo se tem
> 
> $$\begin{align*}
\left[A\right] & = \left(\left[H_{p_t-1,p_t}\right]^{T}\left[D_{p_t}\right]^{T}\right) \otimes \left[H_{p_x, p_x}\right] \\ 
& + \mu \left[H_{p_t, p_t}\right] \otimes \left(\left[D_{p_x}\right] \left[H_{p_x-1,p_x-1}\right] \left[D_{p_x}\right]^{T}\right) \\
& - \mu \left[H_{p_t, p_t}\right] \otimes \left( \left[X_{bound}\right] \left[D_{p_x}\right]^{T} \right)
\end{align*}$$

Já para $B$, temos

> $$B_{m, i} = \int_{\Omega} N_{m,p_t}N_{i,p_x} f \ d\Omega$$
> 
> Nós aproximamos $f(t, x)$ por uma Superfície BSpline
> 
> $$g(t, x) = \sum_{r=0}^{n_t-1}\sum_{s=0}^{n_x-1}N_{r,p_t}(t) \cdot N_{s,p_x}(x) \cdot G_{r,s} \approx f(t, x)$$
> 
> Então encontramos em cada ponto:
> 
> $$f_{m,i} = f(t_m, x_i) = g(t_m, \ x_i) = \sum_{r=0}^{n_t-1}\sum_{s=0}^{n_x-1} N_{r,p_t}(t_m) N_{s,p_x}(x_i) G_{r,s} = \left[N_{p_{t}}(t_m)\right] \left[G\right] \left[N_{p_x}(x_i)\right]$$
> $$\left[G\right] = \left[N_{p_x}(\mathbf{x})\right]^{-1} \left[f\right] \left[N_{p_x}(\mathbf{x})\right]^{-T}$$
> 
> Então aplicando em $B$
> $$\begin{align*}B_{m, i} & = \int_{\Omega} N_{m,p_t}N_{i,p_x} \sum_{r=0}^{n_t-1}\sum_{s=0}^{n_x-1}N_{r,p_t}(t) N_{s,p_x}(x)G_{r,s} \ d\Omega \\ & = \sum_{r=0}^{n_t-1} \sum_{s=0}^{n_x-1} \left(\int_\Omega  N_{m,p_t}N_{i,p_x}N_{r,p_t} N_{s,p_x} \ d\Omega \right)  G_{r,s} \\ & = \sum_{r=0}^{n_t-1} \sum_{s=0}^{n_x-1} \left(\int_{0}^{1} N_{m,p_t}N_{r,p_t} \ dt\right)\left(\int_{0}^{1} N_{i,p_x} N_{s,p_x} \ dx\right) G_{r,s}\\ & = \sum_{r=0}^{n_t-1} \sum_{s=0}^{n_x-1} \left[H_{p_t,p_t}\right]_{m,r} \left[H_{p_x, p_x}\right]_{i,s} \cdot G_{r,s} \\ & = \left( \left[H_{p_t,p_t}\right] \left[G\right] \left[H_{p_x, p_x}\right]^{T}\right)_{m,i}\end{align*}$$
> Ou seja, como resultado final tem-se
> $$\left[B\right] = \left[H_{p_t,p_t}\right] \left[G\right] \left[H_{p_x, p_x}\right]^{T}$$

Agora vamos calcular

### Exercício 1

> $$\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \text{on} \ \Omega$$
>
> No domínio 
> 
> $$\Omega = \underbrace{\left[0, \ 1\right]}_{x} \times \underbrace{\left[0, \ 1\right]}_{t}$$
> 
> E condições de contorno
> 
> $$T(0, \ t) = T(1, \ t) = 0 \ \ \ \ \ \forall 0 \le t$$
> 
> $$T(x, \ 0) = \sin \left(\pi x\right) \ \ \ \ 0 \le x \le 1$$
> 
> A solução exata é dada por
> 
> $$T_{exact}(x, \ t) = \exp \left(-\pi^2 t\right) \cdot \sin \left(\pi x\right)$$

Então encontramos a solução numérica

In [ ]:
def Texact(t, x):
    return np.exp(-np.pi**2 * t) * np.sin(np.pi*x)
nt, nx = 33, 33
pt, px = 4, 5
# Ut = nurbs.GeneratorKnotVector.uniform(pt, nt)
Ut = nurbs.GeneratorKnotVector.uniform(pt, nt)
Ux = nurbs.GeneratorKnotVector.uniform(px, nx)
Nt = nurbs.SplineBaseFunction(Ut)
Nx = nurbs.SplineBaseFunction(Ux)
xplot = np.linspace(0, 1, 1025)
tplot = np.linspace(0, 1, 1025)

Tvals = np.empty((nt, nx), dtype="object")
Tvals[:, 0] = 0
Tvals[:, -1] = 0
Tvals[0, :] = np.linalg.lstsq(Nx(xplot).T, np.sin(np.pi*xplot), rcond=None)[0]

Hptpt = getH(pt, pt, Nt)
Hpxpx = getH(px, px, Nx)
Hpx1px1 = getH(px-1, px-1, Nx)
Hpt1pt = getH(pt-1, pt, Nt)
Dpt = getD(pt, Ut)
Dpx = getD(px, Ux)
Xbound = np.tensordot(Nx[:, px](1), Nx[:, px-1](0), axes=0)
Xbound -= np.tensordot(Nx[:, px](0), Nx[:, px-1](0), axes=0)
A = np.tensordot(Hpt1pt.T @ Dpt.T, Hpxpx, axes=0)
A += np.tensordot(Hptpt, Dpx @ Hpx1px1 @ Dpx.T, axes=0)
A -= np.tensordot(Hptpt, Xbound @ Dpx.T, axes=0)
B = np.zeros((nt, nx), dtype="float64")
Tvals, _ = solve_system(A, B, Tvals)

In [ ]:
xplot = np.linspace(0, 1, 129)
tplot = np.linspace(0, 1, 129)

femvalues = Nt(tplot).T @ Tvals @ Nx(xplot)
exavalues = np.tensordot(np.exp(-np.pi**2 * tplot), np.sin(np.pi*xplot), axes=0)
fig, axis = plt.subplots(1, 3, figsize=(15, 5))
plot_field(xplot, tplot, exavalues, axis[0])
plot_field(xplot, tplot, femvalues, axis[1])
plot_field(xplot, tplot, np.abs(femvalues-exavalues), axis[2])